# Alman Kredi Riskini Tahmin Etme

# 1) Verinin Keşfi (EDA)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
risk = pd.read_csv("../input/german-credit-data-with-risk/german_credit_data.csv", index_col = "Unnamed: 0")
df = risk.copy()

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isnull().sum()

## 1.1) Değişkenlerin İncelenmesi

In [ ]:
df["Job"].value_counts().sort_index()

In [ ]:
df["Housing"].value_counts()

In [ ]:
df["Saving accounts"].value_counts()

In [ ]:
df["Checking account"].value_counts()

In [ ]:
df["Purpose"].value_counts()

In [ ]:
df["Risk"].value_counts()

In [ ]:
df.describe().T

In [ ]:
df.corr()

# 2) Age ve Duration Değişkenlerinin Kategorikleştirilmesi

In [ ]:
for i in df["Age"]:
    if i < 25:
        df["Age"] = df["Age"].replace(i,"0-25")
    elif (i >= 25) and (i < 30):
        df["Age"] = df["Age"].replace(i,"25-30")
    elif (i >= 30) and (i < 35):
        df["Age"] = df["Age"].replace(i,"30-35")
    elif (i >= 35) and (i < 40):
        df["Age"] = df["Age"].replace(i,"35-40")
    elif (i >= 40) and (i < 50):
        df["Age"] = df["Age"].replace(i,"40-50")
    elif (i >= 50) and (i < 76):
        df["Age"] = df["Age"].replace(i,"50-75")

In [ ]:
for i in df["Duration"]:
    if i < 12:
        df["Duration"] = df["Duration"].replace(i,"0-12")
    elif (i >= 12) and (i < 24):
        df["Duration"] = df["Duration"].replace(i,"12-24")
    elif (i >= 24) and (i < 36):
        df["Duration"] = df["Duration"].replace(i,"24-36")
    elif (i >= 36) and (i < 48):
        df["Duration"] = df["Duration"].replace(i,"36-48")
    elif (i >= 48) and (i < 60):
        df["Duration"] = df["Duration"].replace(i,"48-60")
    elif (i >= 60) and (i <= 72):
        df["Duration"] = df["Duration"].replace(i,"60-72")

In [ ]:
df["Job"] = pd.Categorical(df["Job"], categories = [0,1,2,3], 
                           ordered = True)
df["Age"] = pd.Categorical(df["Age"], 
                           categories = ['0-25','25-30', '30-35','35-40','40-50','50-75'], 
                           ordered = True)
df["Duration"] = pd.Categorical(df["Duration"], 
                                categories = ['0-12','12-24', '24-36','36-48','48-60','60-72'], 
                                ordered = True)

# 3) Görselleştirme

## 3.1) Değişkenlerin Görselleştirilmesi

In [ ]:
fig,ax = plt.subplots(ncols = 2, nrows = 3,figsize = (15, 10))
cat_list = ["Age", "Sex", "Job","Housing", "Purpose", "Risk"]
palette = ["crimson", "dodgerblue", "fuchsia", "lime","yellow", "sandybrown"]
count = 0
for i in range(3):
    for j in range(2):
        sns.countplot(df[cat_list[count]], ax = ax[i][j],
                      palette = sns.dark_palette(palette[count],
                                                 reverse=True))
        ax[i][j].set_xticklabels(ax[i][j].get_xticklabels(),rotation = 30)
        count += 1

In [ ]:
plt.figure(figsize = (16, 5))
sns.stripplot(x = "Age", y = "Credit amount", data = df)

In [ ]:
plt.figure(figsize = (16, 5))
sns.boxplot(x = "Age", y = "Credit amount",data = df)

In [ ]:
fig, ax = plt.subplots(ncols = 2, nrows = 2, figsize=(16, 7))
liste = ["Sex", "Risk"]
for i in range(2):
    count = 0
    sns.pointplot(x = "Age", y = "Credit amount", hue = liste[i], data = df, 
                  ax = ax[0][i], palette = ["#FFAE00","#000000"], ci = None)
    sns.pointplot(x = "Purpose", y = "Credit amount", hue = liste[i], data = df, 
                  ax = ax[1][i], palette = ["#FF00F0","#04FF00"], ci = None)
    ax[1][i].set_xticklabels(ax[1][i].get_xticklabels(), rotation = 40, size = 11)
    ax[i][count].legend(loc = "upper left")
    count += 1

## 3.2) Missing Value

In [ ]:
import missingno as msno

In [ ]:
msno.bar(df)

In [ ]:
msno.heatmap(df)

In [ ]:
msno.matrix(df)

# 4) Eksik Değerlerin Doldurulması

In [ ]:
df.isnull().sum()

## 4.1) Saving accounts

In [ ]:
df["Saving accounts"].value_counts()

In [ ]:
df["Saving accounts"].fillna(df["Saving accounts"].mode()[0], inplace = True)

In [ ]:
df["Saving accounts"].value_counts()

## 4.2) Checking account

In [ ]:
df["Checking account"].value_counts()

In [ ]:
df["Checking account"].fillna(df["Checking account"].mode()[0], inplace = True)

In [ ]:
df["Checking account"].value_counts()

In [ ]:
df.isnull().sum()

# 5) Lojistik Regresyon Modeli

In [ ]:
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
df

In [ ]:
df_fit = df.apply(preprocessing.LabelEncoder().fit_transform)

In [ ]:
df_fit

In [ ]:
df_fit.info()

In [ ]:
y = df_fit["Risk"]
X = df_fit.drop(["Risk"], axis=1)

## 5.1) statsmodels

In [ ]:
loj = sm.Logit(y, X)
loj_model= loj.fit()
loj_model.summary()

## 5.2) scikit-learn

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
loj = LogisticRegression(solver = "liblinear")
loj_model = loj.fit(X,y)
loj_model

In [ ]:
loj_model.intercept_

In [ ]:
loj_model.coef_

## 5.3) Tahmin & Model Tuning

In [ ]:
y_pred = loj_model.predict(X)

In [ ]:
confusion_matrix(y, y_pred)

In [ ]:
accuracy_score(y, y_pred)

In [ ]:
print(classification_report(y, y_pred))

In [ ]:
loj_model.predict(X)[0:10]

In [ ]:
loj_model.predict_proba(X)[0:10][:,0:2]

In [ ]:
y[0:10]

In [ ]:
y_probs = loj_model.predict_proba(X)
y_probs = y_probs[:,1]

In [ ]:
y_probs[0:10]

In [ ]:
y_pred = [1 if i > 0.5 else 0 for i in y_probs]

In [ ]:
y_pred[0:10]

In [ ]:
confusion_matrix(y, y_pred)

In [ ]:
accuracy_score(y, y_pred)

In [ ]:
print(classification_report(y, y_pred))

In [ ]:
logit_roc_auc = roc_auc_score(y, loj_model.predict(X))

fpr, tpr, thresholds = roc_curve(y, loj_model.predict_proba(X)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='AUC (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Oranı')
plt.ylabel('True Positive Oranı')
plt.title('ROC')
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.30, 
                                                    random_state = 42)

In [ ]:
loj = LogisticRegression(solver = "liblinear")
loj_model = loj.fit(X_train,y_train)
loj_model

In [ ]:
accuracy_score(y_test, loj_model.predict(X_test))

In [ ]:
cross_val_score(loj_model, X_test, y_test, cv = 10).mean()

# 6) Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
y = df_fit["Risk"]
X = df_fit.drop(["Risk"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.30, 
                                                    random_state=42)

In [ ]:
nb = GaussianNB()
nb_model = nb.fit(X_train, y_train)
nb_model

In [ ]:
nb_model.predict(X_test)[0:10]

In [ ]:
nb_model.predict_proba(X_test)[0:10]

In [ ]:
y_pred = nb_model.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
cross_val_score(nb_model, X_test, y_test, cv = 10).mean()

# 7) Gradient Boosting Machines 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
y = df_fit["Risk"]
X = df_fit.drop(["Risk"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.30, 
                                                    random_state=42)

In [ ]:
gbm_model = GradientBoostingClassifier().fit(X_train, y_train)

In [ ]:
y_pred = gbm_model.predict(X_test)
accuracy_score(y_test, y_pred)

## 6.1) Model Tuning

In [ ]:
from sklearn.model_selection import learning_curve, GridSearchCV

In [ ]:
gbm_model

In [ ]:
gbm_params = {"learning_rate" : [0.001, 0.01, 0.1, 0.05],
             "n_estimators": [100,500,100],
             "max_depth": [3,5,10],
             "min_samples_split": [2,5,10]}

In [ ]:
gbm = GradientBoostingClassifier()

gbm_cv = GridSearchCV(gbm, gbm_params, cv = 10, n_jobs = -1, verbose = 2)

In [ ]:
gbm_cv.fit(X_train, y_train)

In [ ]:
print("En iyi parametreler: " + str(gbm_cv.best_params_))

In [ ]:
gbm = GradientBoostingClassifier(learning_rate = 0.01, 
                                 max_depth = 5,
                                min_samples_split = 10,
                                n_estimators = 100)

In [ ]:
gbm_tuned =  gbm.fit(X_train,y_train)

In [ ]:
y_pred = gbm_tuned.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
cross_val_score(gbm_model, X_test, y_test, cv = 10).mean()